In [1]:
import speech_recognition as sr
from gtts import gTTS
import transformers
from transformers import pipeline
import time
import os
import datetime
import numpy as np
import pandas as pd

In [9]:
from tensorflow.keras import optimizers
from keras.models import model_from_json
import librosa
import librosa.display
import wave

In [3]:
import sys
import warnings
import IPython.display as ipd

In [4]:
qa_model = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
sad_file = open("sad.txt","r")
sad_corpus = sad_file.read()

angry_file = open("angry.txt","r")
angry_corpus = angry_file.read()

happy_file = open("happy.txt","r")
happy_corpus = happy_file.read()

fear_file = open("fear.txt","r")
fear_corpus = fear_file.read()

others_file = open("others.txt","r")
others_corpus = others_file.read()

In [10]:
json_file = open('model_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Emotion_Model.h5")
print("Loaded model from disk")
# the optimiser
opt = optimizers.RMSprop(learning_rate=0.00001, decay=1e-6)
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Loaded model from disk


In [11]:
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
    
    def pre_pro(audio):        
        X, sample_rate = librosa.load(audio
                              ,res_type='kaiser_fast'
                              ,duration=2.5
                              ,sr=44100
                              ,offset=0.5
                             )

        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
        newdf = pd.DataFrame(data=mfccs).T
        
        newdf= np.expand_dims(newdf, axis=2)
        newpred = loaded_model.predict(newdf, 
                         batch_size=16, 
                         verbose=1)
        li = ["angry","fear", "happy","sad","angry","fear","happy","sad"]
        final = newpred.argmax(axis=1)
        final = final.astype(int).flatten()
        return li[final[0]] 
        
    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            
            print("Listening...")
            audio = recognizer.adjust_for_ambient_noise(mic)
            audio = recognizer.listen(mic)

            with open("output.wav", "wb") as f:
                f.write(audio.get_wav_data())
            
            self.emo = ChatBot.pre_pro("output.wav")
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
            print("Emotion:",self.emo)
        except:
            print("Me  -->  ERROR")
    @staticmethod
    def text_to_speech(text):     
        print("Dev --> ", text)              
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        statbuf = os.stat("res.mp3")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
        os.system('start res.mp3')  #if you are using mac->afplay or else for windows->start
        # os.system("close res.mp3")
        time.sleep(int(50*duration))
        os.remove("res.mp3")
    def wake_up(self, text):
        return True if self.name in text.lower() else False
    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M')
    def action_date():
        today = date.today()
        return today

In [24]:
# Running the AI
if __name__ == "__main__":
    ai = ChatBot(name="dev")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex=True
    while ex:
        ai.speech_to_text()
        ## wake up
        if ai.wake_up(ai.text) is True:
            res = "Hello I am Dev the AI, what can I do for you?"
        ## action time
        elif "date" and "yesterday" in ai.text:
            res = ai.action_date()
        elif "time" in ai.text:
            res = ai.action_time()
            
        elif any(i in ai.text for i in ["thank","thanks"]):
            res = np.random.choice(["you're welcome!","anytime!","no problem!","cool!","I'm here if you need me!","mention not"])
        elif any(i in ai.text for i in ["exit","close"]):
            res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","peace out!"])
            ex=False
            
        ## conversation
        else:   
            if ai.text=="ERROR":
                res="Sorry, come again?"
            else:
                question_answerer = pipeline("question-answering")
                question = "how to reply for " + ai.text
                
                context1 = others_corpus
                if(ai.emo=="happy"):
                    context1 = happy_corpus
                elif(ai.emo=="sad"):
                    context1 = sad_corpus
                elif(ai.emo=="angry"):
                    context1 = angry_corpus
                elif(ai.emo=="fear"):
                    context1 = fear_corpus
                    
                reply=question_answerer(question=question, context=context1)
                if(reply['score'] < 0.0001):
                    res="Sorry, come again?"
                else:
                    res = reply['answer']
                
        ai.text_to_speech(res)
    print("----- Closing down Dev -----")

----- Starting up dev -----
Listening...
1/1 [==============================] - 0s 24ms/step
Me  -->  hi Dev nice to meet you
Emotion: sad
Dev -->  Hello I am Dev the AI, what can I do for you?
Listening...
1/1 [==============================] - 0s 24ms/step


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Me  -->  I don't know what to do with my life
Emotion: fear
Dev -->  it's important to communicate your thoughts and feelings in a respectful manner
Listening...
1/1 [==============================] - 0s 23ms/step


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Me  -->  I feel like
Emotion: sad
Dev -->  "I'm here for you. You're not alone
Listening...


KeyboardInterrupt: 